# Workone_resnet:使用Fine_Tuning实现FashionMnist识别准确率达90%以上

In [2]:
import time

import torch
from torchvision.datasets import FashionMNIST
from torchvision import transforms,models
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import lr_scheduler
import os,copy
from tqdm import tqdm

This project use ResNet18 to deal with FashionMnist Dataset

## Load FashionMnist Dataset

In [3]:
data_transforms = {
    'train':transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.2860, ], [0.3205, ])
    ]),
    'val':transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.2860, ], [0.3526, ])
    ])
}

data = {
    x:FashionMNIST(root='dataset',
                   train=True if x =='train' else False,
                   transform=data_transforms[x])
    for x in ['train','val']
}

In [3]:
def Compute_std_and_mean(data):
    mean = 0.0
    std = 0.0
    for X,y in data:
        x = X.view(-1,28*28)
        std += x.std().item()
        mean += x.mean().item()
    mean /= len(data)
    std /= len(data)
    return mean,std

In [4]:
# mean,std = Compute_std_and_mean(data['train'])
# print(mean,std)

In [21]:
batch_size = 64
data_loader ={
    x:DataLoader(data[x],
                 batch_size=batch_size,
                 shuffle=False)
    for x in ['train','val']
}


data_size = {x:len(data[x]) for x in ['train','val']}
print(data_size)
class_names = data['train'].classes

{'train': 60000, 'val': 10000}


## Build Model

In [22]:
class FashionResNet18(nn.Module):
    def __init__(self,num_classes,fix=False):
        super(FashionResNet18, self).__init__()
        self.conv = models.resnet18(pretrained=True)
        if fix:
            for param in self.conv.parameters():
                param.requires_grad = False
        num_fci = self.conv.fc.in_features
        self.conv.fc = nn.Linear(num_fci,num_classes)
    def forward(self,x):
        out = self.conv(x)
        return out

## Train

In [23]:
def train_model(model,loss_fn,optimizer,scheduler=None,num_epochs = 2):
    best_model_dict = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch: {epoch:^5d}/{num_epochs-1:^5d}')
        print("----------------------------------------------")
        for state in ['train','val']:
            if state == 'train':
                model.train()
                if scheduler is not None:
                    scheduler.step()
            else:
                model.eval()

            total_loss = 0.0
            total_correct = 0

            for i,(X,y) in enumerate(tqdm(data_loader[state])):
                time.sleep(0.01)
                X = torch.cat([X,X,X],1).to(device)
                y = y.to(device)#[64]

                optimizer.zero_grad()

                with torch.set_grad_enabled(state =='train'):
                    outputs = model(X)#[64,10]
                    loss = loss_fn(outputs,y)
                    pred = outputs.argmax(1)#[64]

                    if state == 'train':
                        loss.backward()
                        optimizer.step()

                total_loss += loss.item() * X.shape[0]
                total_correct += (pred == y).sum().item()

                if i%10 == 0:
                    print(f"Loss:{loss}")

            acc = total_correct/data_size[state]
            if state == 'val' and acc > best_acc:
                best_acc  = acc
                best_model_dict = copy.deepcopy(model.state_dict())
            print(f"{state} --- Loss:{total_loss/data_size[state]} "
                  f"Acc:{acc}")
    return best_model_dict



In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'------------ Use Device: {device} ------------')


------------ Use Device: cuda ------------


In [54]:
model = FashionResNet18(len(class_names)).to(device)

In [55]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.015,momentum=0.9,nesterov=True)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

In [56]:
epochs = 5

In [57]:
best_model_dict = train_model(model,loss_fn,optimizer,exp_lr_scheduler,epochs)

E:\Anaconda\envs\torch1.10\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch:   0  /  4  
----------------------------------------------


  0%|▏                                                                                 | 2/938 [00:00<01:23, 11.21it/s]

Loss:2.523014783859253


  1%|█                                                                                | 12/938 [00:00<01:15, 12.30it/s]

...

Loss:0.1736031323671341


 97%|█████████████████████████████████████████████████████████████████████████████▉  | 153/157 [00:07<00:00, 21.33it/s]

Loss:0.18470148742198944


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:07<00:00, 20.77it/s]

val --- Loss:0.2531430483818054 Acc:0.9144


## Save Model

In [58]:
def save_model(model_path):
    root_dir = 'model'
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    model_dir = os.path.join(root_dir,model_path)
    if not os.path.exists(model_path):
        torch.save(best_model_dict,model_dir)
    print("Successfully save!")

In [59]:
save_model("FashionMnistNesterov.pth")

Successfully save!


## Load Model

In [60]:
def load_model(model,model_pth):
    model = model
    model.load_state_dict(torch.load(model_pth))
    dataloader = DataLoader(data['val'],batch_size=batch_size,shuffle=False)
    model.to(device)
    model.eval()
    correct = 0.0
    with torch.no_grad():
        for X,y in dataloader:
            X = torch.cat([X,X,X],1)
            X,y = X.to(device),y.to(device)
            pred = model(X)
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
        size = len(dataloader.dataset)
        print(f"Accuary:{correct*100.0/size:>4f}%")



In [61]:
load_model(FashionResNet18(10),'model/FashionMnistNesterov.pth')

Accuary:91.440000%
